In [ ]:
import os
import pandas as pd

# local path and filenames
path = '../../data/raw'
tags = ['2019', '2021', '2022', '2023', '2024']
files = {
    '2024':'HMS_2024-2025_PUBLIC_instchars.csv',
    '2023':'HMS_2023-2024_PUBLIC_instchars.csv',
    '2022':'HMS_2022-2023_PUBLIC_instchars.csv',
    '2021':'HMS_2021-2022_PUBLIC_instchars.csv',
    '2019':'HMS_2019-2020_PUBLIC_instchars.csv'
}

# load all the dataframes with names df1, df2, ... df5
for i, tag in enumerate(tags):

    full_path = os.path.join(path, files[tag])

    df_name = f"df{i+1}"
    globals()[df_name] = pd.read_csv(full_path, low_memory=False)

#### Methods for certain tasks

In [1]:
# method: drop all columns after a certain column number, or with certain names
def drop_columns_after(dataframe, col_num, drop_names=None):
    cols_to_keep = dataframe.columns[:col_num+1]
    result = dataframe.loc[:, cols_to_keep]
    if drop_names:
        result = result.drop(columns=[col for col in drop_names if col in result.columns])
    return result

# method: print features alphatebatically sorted after the first two columns
def coloumn_sorter(tags):

    for idx in range(1, 6):
        df = globals()[f'df{idx}_new']

        # sort all columns after the first two
        cols = df.columns.tolist()
        fixed_cols = cols[:2]
        sorted_cols = sorted(cols[2:])
        globals()[f'df{idx}_new'] = df[fixed_cols + sorted_cols]

        print(tags[idx-1])
        print(df.columns.tolist())
        print('-' * 40)

### With respect to each file, remove all features except gender and sexual orientation

In [3]:
# remove all columns except gender and sexual orientation
df1_new = drop_columns_after(df1, 16, ['StartDate', 'RecordedDate', 'survey', 'nrweight'])
df2_new = drop_columns_after(df2, 24, ['startdate', 'recordeddate', 'survey', 'nrweight'])
df3_new = drop_columns_after(df3, 22)
df4_new = drop_columns_after(df4, 23, ['StartDate', 'RecordedDate'])
df5_new = drop_columns_after(df5, 25, ['StartDate', 'Finished', 'RecordedDate', 'schoolnum'])

### Add will_remove and fix age column 

In [ ]:
# add will_remove = 1 for age > 60
for idx in range(1, 6):
    df = globals()[f'df{idx}_new']

    # clean for age > 70
    df.insert(1, 'will_remove', None)
    df.loc[df['age'].isna(), 'will_remove'] = 1
    # flag ages that are not whole numbers
    df.loc[(df['age'] - df['age'].round()).abs() > 1e-8, 'will_remove'] = 1
    df.loc[df['age'] > 60, 'will_remove'] = 1

### Fix gender columns

In [ ]:
for idx in range(1, 6):
    df = globals()[f'df{idx}_new']

    # clean for text responses in gender and other gender columns
    if 'gender_text' in df.columns:
        df.loc[df['gender_text'].notna(), 'will_remove'] = 1
        df.drop(columns=['gender_text'], inplace=True)
    if 'gender_selfID' in df.columns:
        df.drop(columns=['gender_selfID'], inplace=True)
    if 'gender_selfid' in df.columns:
        df.drop(columns=['gender_selfid'], inplace=True)
    if 'gender_prefnoresp' in df.columns:
        df.loc[df['gender_prefnoresp'].notna(), 'will_remove'] = 1
        df.drop(columns=['gender_prefnoresp'], inplace=True)

    # merge queer and nonbin into one column called nonbin (using logical OR)
    if 'gender_queer' in df.columns and 'gender_nonbin' in df.columns:
        df['gender_nonbin'] = df[['gender_queer', 'gender_nonbin']].max(axis=1)
        df.drop(columns=['gender_queer'], inplace=True)

    if 'gender_transf' in df.columns and 'gender_transm' in df.columns:
        df['gender_trans'] = df[['gender_transf', 'gender_transm']].max(axis=1)
        df.drop(columns=['gender_transf', 'gender_transm'], inplace=True)

In [6]:
# remap the number in df1_new and add will_remove = 1 for gender = 5
df1_new['gender'] = df1_new['gender'].replace({4: 3, 5: 4, 6: 5, 2:7, 1:2})
df1_new['gender'] = df1_new['gender'].replace({7: 1})

df1_new.loc[df1_new['gender'] == 5, 'will_remove'] = 1
df1_new.loc[df1_new['gender'].isna(), 'will_remove'] = 1

In [7]:
# for df2_new to df5_new, create a new gender column based on the multiple binary columns
for idx in range(2, 6):  # 2021-2024 are df2_new to df5_new
    df = globals()[f'df{idx}_new']
    gender_cols = ['gender_female', 'gender_male', 'gender_trans', 'gender_nonbin']

    def get_gender(row):
        vals = [row.get(col, 0) for col in gender_cols]
        count = sum([v == 1.0 for v in vals])
        if count > 1 or count == 0:
            return 5
        if row.get('gender_female', 0) == 1.0:
            return 1
        if row.get('gender_male', 0) == 1.0:
            return 2
        if row.get('gender_trans', 0) == 1.0:
            return 3
        if row.get('gender_nonbin', 0) == 1.0:
            return 4
        return 5

    df['gender'] = df.apply(get_gender, axis=1)
    df.loc[df['gender'].isna(), 'will_remove'] = 1
    df.loc[df['gender'] == 5, 'will_remove'] = 1

    # drop gender columns
    for col in ['gender_female', 'gender_male', 'gender_nonbin', 'gender_trans']:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)

### Fix sex_at_birth column

In [8]:
for idx in range(1, 6):
    df = globals()[f'df{idx}_new']

    # Set will_remove = 1 if sex_birth is missing
    df.loc[df['sex_birth'].isna(), 'will_remove'] = 1
    # Set sex_birth to NaN where sex_birth == 4
    df.loc[df['sex_birth'].isna(), 'sex_birth'] = 4

In [9]:
for idx in range(1, 6):
    df = globals()[f'df{idx}_new']

    # clean for text responses in gender
    if 'sexual_text' in df.columns:
        df.loc[df['sexual_text'].notna(), 'will_remove'] = 1
        df.drop(columns=['sexual_text'], inplace=True)
    if 'sexual_selfID' in df.columns:
        df.drop(columns=['sexual_selfID'], inplace=True)
    if 'sexual_selfid' in df.columns:
        df.drop(columns=['sexual_selfid'], inplace=True)
    if 'q2_5_8' in df.columns:
        df.drop(columns=['q2_5_8'], inplace=True)
    if 'q2_6_5' in df.columns:
        df.drop(columns=['q2_6_5'], inplace=True)

In [10]:
for idx in range(1,6):
    df = globals()[f'df{idx}_new']
    sexual_cols = ['sexual_asexual', 'sexual_bi', 'sexual_g', 'sexual_h', 'sexual_l',
                   'sexual_pan', 'sexual_prefnoresp', 'sexual_queer', 'sexual_quest',
                   'seuxal_prefnoresp', 'sexual_other']

    def get_sexual_pref(row):
        vals = [row.get(col, 0) for col in sexual_cols]
        count = sum([v == 1.0 for v in vals])

        if count > 1 or count == 0:
            return 7
        if row.get('sexual_h', 0) == 1.0:
            return 1
        if row.get('sexual_l', 0) == 1.0:
            return 2
        if row.get('sexual_g', 0) == 1.0:
            return 3
        if row.get('sexual_bi', 0) == 1.0:
            return 4
        if row.get('sexual_queer', 0) == 1.0:
            return 5
        if row.get('sexual_quest', 0) == 1.0:
            return 6       
        return 7

    df['sexual_pref'] = df.apply(get_sexual_pref, axis=1)

    df.loc[df['sexual_pref'] == 7, 'will_remove'] = 1

    # drop gender columns
    for col in sexual_cols:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)

In [ ]:
for idx, tag in enumerate(tags):
    df = globals()[f'df{idx+1}_new']
    df['will_remove'] = df['will_remove'].fillna(0)
    out_name = f"../../data/interim/{tag}-{int(tag)+1}_saaransh.csv"
    df.to_csv(out_name, index=False)